In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# import the path
import os
path = os.path.abspath(os.path.join("..", "08_portfolio_management", "portfolio_data.csv"))

# read the CSV file
df = pd.read_csv(path)

# show the first few rows of the dataframe
df.head()

,Ticker,Last_Price,Quantity,Position_Value,Position_Weight
0,AAPL,232.560,18.00,4186.080,0.981974
1,AMD,168.580,18.00,3034.440,0.711821
2,ASML,654.300,6.00,3925.800,0.920917
3,BNB,857.400,13.09,11223.366,2.632785
4,BR50,21.855,213.00,4655.115,1.092000
